Imports

In [ ]:
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

from src.get_options import get_eval_config_from_options
from src.model_creation import ModelCreationConfig
from src.utils import select_best_device
from src.eval import eval_from_config
from src.utils import hf_download_and_unjson

Args

In [ ]:
''' Model '''
## Specify EITHER model_path_or_repo and (optionally) lora_path_or_repo OR a path or repo which contains a model_creation_config.json
model_path_or_repo: str | None = "Lapisbird/Llama-adaLR-model-latent-6"
lora_path_or_repo: str | None = None
## OR
mcc_path_or_repo: str | None = None

''' Eval '''
batch_size: int = 128
eval_options: str = "gsm8k-aug-test/latent_thinking"

Get model and eval config

In [ ]:
if model_path_or_repo is not None and mcc_path_or_repo is not None:
    raise ValueError("Specify ONE of model_path_or_repo and mcc_path_or_repo")

if model_path_or_repo is not None:
    mcc = ModelCreationConfig(path_or_repo=model_path_or_repo, is_latent="latent" in eval_options or "latent" in model_path_or_repo, is_lora=bool(lora_path_or_repo), lora_path_or_repo=lora_path_or_repo)
elif mcc_path_or_repo is not None:
    mcc_path = Path(mcc_path_or_repo) / "model_creation_config.json"
    if mcc_path.exists():
        mcc = ModelCreationConfig.load_from_json(mcc_path)
    else:
        mcc = ModelCreationConfig(
            **hf_download_and_unjson(mcc_path_or_repo + "/model_creation_config.json")
        )
else:
    raise ValueError("Not one of model_path_or_repo or mcc_path_or_repo provided")
    
model, tokenizer = mcc.get_model_and_tokenizer()

eval_config = get_eval_config_from_options(eval_options)

device = select_best_device("m")
print("Using device: ", device)
model.to(device)

print()

Test

In [ ]:
results = eval_from_config(model, tokenizer, eval_config, batch_size=batch_size, return_decoded=True)
results